In [11]:
!pip install langchain langchain_community langchain_openai chromadb pymupdf nest_asyncio --quiet

from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough
)
from langchain.schema.output_parser import StrOutputParser

!pip install PyMuPDF pytesseract Pillow --quiet
!pip install tokenizers
!pip install pymupdf

In [2]:
import pytesseract

print(pytesseract)

import chromadb
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader

<module 'pytesseract' from 'C:\\Users\\aswin\\anaconda3\\Lib\\site-packages\\pytesseract\\__init__.py'>


In [76]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

import fitz  # PyMuPDF
from PIL import Image
import pytesseract

# Define the path to the PDF file
pdf_path = r'C:\Users\aswin\OneDrive\Desktop\Important\Presentations\ML\Data\Form990\Central_Jersey_College_Prep_2022.pdf'

chromdb_path = './chroma.db'


import os
os.environ["OPENAI_API_KEY"] = 'sk-proj-8ql-ivq_WjkSlk5T3BlbkFJpzHw3wUQNnTJh-PTB1dc9yLb-unj9aOdikOj6PLW5fhUuky0CfMu91Gfp-kR3HjBN8BdoUuwIA'

# create vector store with Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores.utils import filter_complex_metadata # import filter_complex_metadata

# Open the PDF file
pdf_document = fitz.open(pdf_path)
pages=[]
# Iterate through each page
for page_num in range(pdf_document.page_count):
    zoom = 3.5
    mat = fitz.Matrix(zoom, zoom)
    page = pdf_document.load_page(page_num)
    pix = page.get_pixmap(matrix = mat)
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    # Perform OCR on the image
    text = pytesseract.image_to_string(img)
    pages.append(text)
    # Print the extracted text
    #print(f"Page {page_num + 1}:\n{text}\n")
    
# Close the PDF document
pdf_document.close()
print(pages)
# split the pages into chunks
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=50
)
pages_to_persist = text_splitter.create_documents(pages)
pages.extend(pages_to_persist)
#chroma_setting = Settings(anonymized_telemetry=False, is_persistent=True, persist_directory=chromdb_path)
vectordb = Chroma.from_documents(documents=pages_to_persist, embedding=OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"]))#,persist_directory=chromdb_path,client_settings=chroma_setting)
vectordb.persist()
retriever = vectordb.as_retriever()

['Department of the\n\nTreasury\n\nA F r the 2022 calendar year, or tax year beginning 07-01-2022\n\nB Check if applicable:\nO Address change\nO Name change\n\nrom IO\n\nReturn of Organization Exempt From Income Tax\n\nOMB No. 1545-0047\n\nUnder section 501(c), 527, or 4947(a)(1) of the Internal Revenue Code (except private foundations)\n® Do not enter social security numbers on this form as it may be made public.\n\nC Name of organization\nCENTRAL JERSEY COLLEGE PREP\nCHARTER SCHOOL\n\n» Go to www. irs.gov/Form990 for instructions and the latest information.\n\n, and ending 06-30-2023\n\nOpen to Public\nInspection\n\nD Employer identification number\n\n20-4477065\n\nDoing business as\n\n1 Initial return\n\n1 Final return/terminated\n0 Amended return\n\nDO Application pending}\n-\n\nE Telephone number\n\nNumber and street (or P.O. box if mail is not delivered to street address)\n101 METTLERS ROAD\n\nRoom/suite\n\n(732) 302-9991\n\nCity or town, state or province, country, and ZIP or fo

In [78]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [80]:
#Creating a RAG Pipeline
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
# RAG
template = """You are an AI language model Form990 Tax assistant.Answer the following question based on this context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
llm = ChatOpenAI(temperature=0, openai_api_key=os.environ["OPENAI_API_KEY"])
final_rag_chain = (
    #{"context": retriever | format_docs, "question": RunnablePassthrough()}

    RunnablePassthrough.assign(
        context=lambda x: format_docs(vectordb.similarity_search(x["question"], k=10)),
    )
    | prompt
    | llm
    | StrOutputParser()
)
    

In [94]:
question="In Part IX Statement of Functional Expenses what is the Total Functional expenses for Management and general expenses for Kingsford High School and compare it with Central_Jersey_College?"

In [96]:
final_rag_chain.invoke({"question":question})

'The Total Functional expenses for Management and general expenses for Kingsford High School is $45,036, while for Central Jersey College Prep Charter School it is $17,232,900. There is a significant difference in the expenses for Management and general expenses between the two organizations.'